In [2]:
#Importaçoes
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from lxml import etree
from selenium import webdriver


In [3]:
#Definicao do webdriver e da url para fazer scrapping
DRIVER_PATH = 'd:\chromedriver\chromedriver.exe'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)

url='https://www.linkedin.com.br/jobs/'

driver.get(url)
time.sleep(2)

In [4]:
#Inserção do tipo de emprego e localização
job_search=driver.find_element_by_xpath('//*[@id="JOBS"]/section[1]/input')
loc_search=driver.find_element_by_xpath('//*[@id="JOBS"]/section[2]/input')

job_search.send_keys("ciência de dados")
loc_search.send_keys("São Paulo")

#Clicar no Botao Search
driver.find_element_by_xpath('/html/body/main/section[1]/section/div[2]/button[2]').click()
time.sleep(2)


In [5]:
# Scroll-Down até ao final da página (nota o numero máximo de anuncios e 50)
lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
                lastCount = lenOfPage
                time.sleep(3)
                lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
                if lastCount==lenOfPage:
                    match=True
time.sleep(2)

In [6]:
# Buscar todos os links do anuncios pesquisado
links_anuncios=[]
list_anuncio=driver.find_elements_by_class_name("result-card__full-card-link")
for anuncio in list_anuncio:
    links_anuncios.append(anuncio.get_attribute('href')[:anuncio.get_attribute('href').find('?refId')]+'/')


In [7]:
# Definição da Dataframe para guardar os dados a serem extraidos
colunas=['Descricao_Geral','Empresa', 'Local','Descricao' ,'nivel_de_experiencia','tipo_trabalho','funcao','setor']
df_jobs = pd.DataFrame(columns=colunas)    





In [8]:
# PAra cada anuncio, buscar a informação da vaga e escrever num dataframe
for link_anuncio in links_anuncios:
    anuncio=[]
    driver.get(link_anuncio)
    time.sleep(3)
    anuncio.append(driver.find_element_by_xpath("/html/body/main/section[1]/section[2]/div[1]/div[1]/h1").text)  #Descricao_Geral
    anuncio.append(driver.find_element_by_xpath("/html/body/main/section[1]/section[2]/div[1]/div[1]/h3[1]/span[1]").text) #Empresa
    anuncio.append(driver.find_element_by_xpath("/html/body/main/section[1]/section[2]/div[1]/div[1]/h3[1]/span[2]").text) #Local
    driver.find_element_by_xpath("/html/body/main/section[1]/section[3]/div/section/button[1]").click() #Link para Exibir mais da Descrição 
    anuncio.append(driver.find_element_by_xpath("/html/body/main/section[1]/section[3]/div/section/div").text) #Descricao
    anuncio.append(driver.find_element_by_xpath("/html/body/main/section[1]/section[3]/ul/li[1]").text.replace('Nível de experiência\n','')) #nivel_de_experiencia
    anuncio.append(driver.find_element_by_xpath("/html/body/main/section[1]/section[3]/ul/li[2]").text.replace('Tipo de trabalho\n','')) #tipo_trabalho
    anuncio.append(driver.find_element_by_xpath("/html/body/main/section[1]/section[3]/ul/li[3]").text.replace('Função\n','')) #funcao
    anuncio.append(driver.find_element_by_xpath("/html/body/main/section[1]/section[3]/ul/li[4]").text.replace('Setores\n','')) #setor
    df_jobs.loc[len(df_jobs), :] = anuncio

In [9]:
#Visulizacao da Extração
df_jobs.head()

,Descricao_Geral,Empresa,Local,Descricao,nivel_de_experiencia,tipo_trabalho,funcao,setor
0,Banco de talentos Diversidade - AMAZON BRASIL,Amazon,"São Paulo, SP","Description\n\nNa Amazon, estamos preocupados ...",Não aplicável,Tempo integral,Administração,SoftwareTecnologia da informação e serviços In...
1,Ciência de Dados,Mindsight,"São Paulo, SP",Quem nós somos\n\nA Mindsight é uma empresa qu...,Não aplicável,Tempo integral,EngenhariaTecnologia da informação,Software
2,Analista de Inteligência de Dados Jr.,MSX International,"São Paulo, SP",Analista de Inteligência de Dados Jr.\n\nA MSX...,Júnior,Tempo integral,OutroTecnologia da informaçãoGerenciamento,Tecnologia da informação e serviços Indústria ...
3,Diversidade - Inclusão da Comunidade Negra,Nubank,"São Paulo, SP",Enfrentar o complexo sistema bancário para cap...,Não aplicável,Tempo integral,Outro,SoftwareInternetServiços financeiros
4,Banco de Talentos,Dextra,"São Paulo, SP","Você tem interesse em trabalhar conosco, mas n...",Assistente,Tempo integral,Recursos humanos,Tecnologia da informação e serviços SoftwareIn...


In [10]:
#Criar arquivo com base no dataframe com os dados
df_jobs.to_csv('jobs.csv',sep=';', encoding="utf-8", index=False)